In [6]:
import numpy as np

# How to solve a system of Diophantine equations

This is a practical guide that explains a method to find all solutions of a system of Diophantine equations $$Cy=c, y \in \mathbb{Z}^m, C \in \mathbb{Q}^{q\times m}, c \in \mathbb{Q}^q,$$
or to return the emptyset if none exists.
It follows an implementation of the algorithms outlined in the Book "Theory Of Linear And Integer Programming"
by Alexander Schrijver [1], and "Integer Programming and Algorithmic Geometry of Numbers", by Friedrich Eisenbrand [2]. 

## The Euclidean algorithm
At the core of solving a system of Diophantine equations is the Euclidean algorithm that is implemented below. In its original version, it finds the greatest common divisor of two rational numbers $\alpha$ and $\beta$. The extension below also memorizes, what combination yields the greatest common divisor, that is, what combination $k_1,k_2$ solves the system $$ k_1 \alpha + k_2 \beta = gcd(\alpha,\beta),$$
and what combination $k_{n_1}$, $k_{n_2}$ yields $$k_{n_1} \alpha + k_{n_2} \beta = 0$$

In [1]:
def euclidean_algorithm(coeff):
    I = np.eye(2)
    A = np.concatenate((np.array([coeff]),I), axis = 0).astype('int')
#    print('Starting with')
#    print(A)
    alpha = A[0,0]
    beta = A[0,1]
    if alpha < 0:
#        print('Mutliplying Column 1 with -1')
        A[:,0] = -A[:,0]
        alpha = -alpha
#        print(A)
    if beta <0:
#        print('Mutliplying Column 2 with -1')
        A[:,1] = -A[:,1]
        beta = -beta
#        print(A)
    while ((alpha != 0) & (beta != 0)):
        if alpha >= beta:
            A[:,0] = A[:,0] - np.floor_divide(alpha,beta)*A[:,1]
#            print("Reducing left column yields: ")
#            print(A)
        else:
            A[:,1] = A[:,1] - np.floor_divide(beta,alpha)*A[:,0]
#            print("Reducing right column yields: ")
#            print(A)
        alpha = A[0,0]
        beta = A[0,1] 
    if A[0,0]==0:
#        print('Switching Columns such that the first column is always the gcd-column')
        b = A[:,0].copy()
        A[:,0] = A[:,1]
        A[:,1] = b
#        print(A)
    return A        

As an example, consider the $(\alpha,\beta) = (18,32)$.

In [93]:
result = euclidean_algorithm([18,32])
result

Starting with
[[18 32]
 [ 1  0]
 [ 0  1]]
Reducing right column yields: 
[[18 14]
 [ 1 -1]
 [ 0  1]]
Reducing left column yields: 
[[ 4 14]
 [ 2 -1]
 [-1  1]]
Reducing right column yields: 
[[ 4  2]
 [ 2 -7]
 [-1  4]]
Reducing left column yields: 
[[ 0  2]
 [16 -7]
 [-9  4]]
Switching Columns such that the first column is always the gcd-column
[[ 2  0]
 [-7 16]
 [ 4 -9]]


array([[ 2,  0],
       [-7, 16],
       [ 4, -9]])

The returned matrix from the above implemented Euclidean algorithm contains all the information we need for finding all solutions to a single diophantine equation of two variables. The nonzero entry of the first row yields the gcd (thus we call the corresponding column gcd-column). The bottom two entries of the gcd-column contain $(k_1,k_2)$, the linear integer combination that yields the gcd. The bottom two entries of the zero-column (the column corresponding to the zero entry in the first row) corresponds to $\eta$, the linear integer combination that yields zero. 

In our example, we immediately see that $2$ is the gcd, that the vector $(-7,4)$ yields the gcd and that any multiple of $(-16,9)$ yields $0$. 

## Solving a linear diophantine equation with two variables

Next, we will use the Euclidean algorithm to solve a linear diophantine equation.

The system $\alpha y_1 + \beta y_2 = b$ has a solution, if and only if $\frac{b}{gcd(\alpha,\beta)} \in \mathbb{Z}$. Due to the relationship $\alpha k_1 + \beta k_2 = gcd(\alpha,\beta)$, we may set $(\bar{y}_1,\bar{y}_2) := \frac{b}{gcd(\alpha,\beta)} (k_1,k_2)$, $\eta := (k_{n_1},k_{n_2})$, and define the set of solutions as

$$ \{y | \hspace{0.1cm} \alpha y_1 + \beta y_2 = b \} = \{y | \hspace{0.1cm} y = \bar y + k \eta, k \in \mathbb{Z}\}.$$ 

This algorithm is implemented below

In [2]:
def solve_diophantine(coeff,b):
    A = euclidean_algorithm(coeff)
    gcd_column = np.argmax(result[0,:])
    zero_column = np.argmin(result[0,:])
    gcd = A[0,gcd_column]    
    if (b % gcd == 0):
        print('System is solvable')
        y_0 = (b / gcd) * A[1:,gcd_column]
        print('y0 = ' + str(y_0))
        eta = A[1:,zero_column]
        print('eta = ' + str(eta))
        return y_0,eta
    else:
        print('System has no sultion')
        return []    

As an example, let us solve the diophantine equation $18 y_1 + 32 y_2 = 6$

In [95]:
a = np.array([18,32])
b = 6
solve_diophantine(a,b)

Starting with
[[18 32]
 [ 1  0]
 [ 0  1]]
Reducing right column yields: 
[[18 14]
 [ 1 -1]
 [ 0  1]]
Reducing left column yields: 
[[ 4 14]
 [ 2 -1]
 [-1  1]]
Reducing right column yields: 
[[ 4  2]
 [ 2 -7]
 [-1  4]]
Reducing left column yields: 
[[ 0  2]
 [16 -7]
 [-9  4]]
Switching Columns such that the first column is always the gcd-column
[[ 2  0]
 [-7 16]
 [ 4 -9]]
System is solvable
y0 = [-21.  12.]
eta = [16 -9]


(array([-21.,  12.]), array([16, -9]))

Thus, all solutions to the system are defined by $y = (-21,12)^\top + k * (16,-9)^\top, k \in \mathbb{Z}$.

If, however, we set $b$ to any number that is not dividable by gcd($\alpha$,$\beta$), the set of solutions is empty, as shown below.

In [28]:
b = 7
solve_diophantine(a,b)

Starting with
[[18 32]
 [ 1  0]
 [ 0  1]]
Reducing right column yields: 
[[18 14]
 [ 1 -1]
 [ 0  1]]
Reducing left column yields: 
[[ 4 14]
 [ 2 -1]
 [-1  1]]
Reducing right column yields: 
[[ 4  2]
 [ 2 -7]
 [-1  4]]
Reducing left column yields: 
[[ 0  2]
 [16 -7]
 [-9  4]]
Switching Columns
[[ 2  0]
 [-7 16]
 [ 4 -9]]
System has no sultion


[]

## Solving a system of linear diophantine equations

Next, we examine a way to find all solutions to a system of $q$ linear diophantine equations and $m$ variables, that is, to find all solutions of
$$Cy=c, y \in \mathbb{Z}^m, C \in \mathbb{Q}^{q\times m}, c \in \mathbb{Q}^q,$$
or to return the emptyset if none exists. We assume that $C$ has full row rank

A Hermite normal form $H$ of a matrix $C$ has the form $$H \in \mathbb{Z}^{q\times m} = [B,0], B\in\mathbb{Q}^{q\times q},$$ where $B$ is a lower triangular, nonnegative matrix in which each row has a unique maximum entry which is located at the main diagonal of $B$. Due to the Hermite normal form Theorem, we know that every rational matrix of full row rank can be brought into Hermite normal form by a series of elementary (unimodular) column operations (exchanging two columns, multiplying a column with -1, adding a multiple of one column to another). These elementary column operations are "memorized" in a matrix $U$, such that we have $AU=H$.

If $B^{-1}b$ is not integral, the System does not have a soltuion (cf. [1,Corollary 5.3b]). Otherwise,
with the matrices $H$ and $U$ at hand, we may compute one solution to the system $Cy=c$ as 
$$ y_0 = U \left(\begin{array}{c}
B^{-1}b \\
0_{[m-q,1]}
\end{array} \right),$$
because we have $$ A y_0 = A U \left(\begin{array}{c}
B^{-1}b \\
0_{[m-q,1]}
\end{array} \right) = [B,0] \left(\begin{array}{c}
B^{-1}b \\
0_{[m-q,1]}
\end{array} \right) = b. $$


The kernel of $C$ may be determined as $$V = U \left(\begin{array}{c}
0_{[q, m-q]} \\
I_{[m-q,m-q]}
\end{array} \right),$$

because $$ AV = A U \left(\begin{array}{c}
0_{[q, m-q]} \\
I_{[m-q,m-q]}
\end{array} \right) = [B,0] \left(\begin{array}{c}
0_{[q, m-q]} \\
I_{[m-q,m-q]}
\end{array} \right) = 0$$


Hence, the challenge of solving the system $Cy=c$ boils down to finding the Hermite normal form $H$ and $U$. Below, we implement [2,Algorithm 14.1], which computs the matrices $H$ and $U$ using Euclids algorithm implemented above as a subroutine. We start with $H:=C$ and, at each iteration, update $H$ and $U$ as $$H = H M, \text{ and } U = U M,$$ where $M$ is the matrix that corresponds to the unimodular column operations that are determined by using Euclids Algorithm.

In [26]:
def hermite_normal_form(C):
    print('Bringing matrix C: ')
    print(C)
    print('To Hermite normal form H \n')
    print('Starting with H:=C and U:=I')
    q,m = np.shape(C)
    U = np.eye(m)
    H = C.copy()
    for i in range(0,q):
        print('\nCreating zeros right to the diagonal')
        for j in range(i+1,m):
            if H[i,j] != 0:
                print("Working with row "+ str(i) + " and columns " + str(i) + "," + str(j))
                udpate_indices = np.array([i,j])
                print('Euclidean Algorithm yields')
                res_eucl = euclidean_algorithm([H[i,i],H[i,j]])
                print(res_eucl)
                ggt = res_eucl[0,0]
                
                #--------------------------------------------------#
                # Only for printing purposes
                update_visualization = np.eye(m)
                update_visualization[i,i] = res_eucl[1,0]
                update_visualization[j,i] = res_eucl[2,0]
                update_visualization[i,j] = res_eucl[1,1]
                update_visualization[j,j] = res_eucl[2,1]
                print('M corresponds to')
                print(update_visualization)
                #--------------------------------------------------#
                
                unimodular_updateH = np.dot(H[:,udpate_indices], res_eucl[1:,:])
                unimodular_updateU = np.dot(U[:,udpate_indices], res_eucl[1:,:])
                H[:,udpate_indices] = unimodular_updateH
                U[:,udpate_indices] = unimodular_updateU
                print('Updated H is:')
                print(H)
                print('Updated U is:')
                print(U)

        print('\nReducing entries left to the diagonal')    
        for j in range(0,i):
            update_indices = np.array([j,i])    # Note that j<i and thus the order [j,i]
            q = np.floor_divide(H[i,j],H[i,i])
            print('Reducing element left of the diagonal with ' + str(q))
            reduction_matrix = np.array([[1,0],[-q,1]])
            print(reduction_matrix)
            print(H[:,update_indices])
            unimodular_reduction_updateH = np.dot(H[:,update_indices],reduction_matrix)
            unimodular_reduction_updateU = np.dot(U[:,update_indices],reduction_matrix)
            H[:,update_indices] = unimodular_reduction_updateH
            U[:,update_indices] = unimodular_reduction_updateU
            print(H)
            print(U)
    print('\n\nHermite normal form of C =')
    print(C)
    print('is H=')
    print(H)
    print('We have CU=H, and U=')
    print(U)
    return H,U

In [4]:
def hermite_normal_form(C):
    q,m = np.shape(C)
    U = np.eye(m)
    H = C.copy()
    for i in range(0,q):
        for j in range(i+1,m):
            if H[i,j] != 0:
                udpate_indices = np.array([i,j])
                res_eucl = euclidean_algorithm([H[i,i],H[i,j]])
                ggt = res_eucl[0,0]              
                unimodular_updateH = np.dot(H[:,udpate_indices], res_eucl[1:,:])
                unimodular_updateU = np.dot(U[:,udpate_indices], res_eucl[1:,:])
                H[:,udpate_indices] = unimodular_updateH
                U[:,udpate_indices] = unimodular_updateU
        for j in range(0,i):
            update_indices = np.array([j,i])    # Note that j<i and thus the order [j,i]
            q = np.floor_divide(H[i,j],H[i,i])
            reduction_matrix = np.array([[1,0],[-q,1]])
            unimodular_reduction_updateH = np.dot(H[:,update_indices],reduction_matrix)
            unimodular_reduction_updateU = np.dot(U[:,update_indices],reduction_matrix)
            H[:,update_indices] = unimodular_reduction_updateH
            U[:,update_indices] = unimodular_reduction_updateU
    return H,U

We will use the matrix 
$$ C = \left(\begin{matrix}
2 & 3 & 4 & -3   \\ 
-2 & 4 & 6 & 7 \\
\end{matrix}\right), \text{ and } c = \left(\begin{array}{c}
9\\
2
\end{array} \right) $$ as an example. Below, we print each step of the algorithm.

In [22]:
C = np.array([[2,3,4,-3],[-2,4,6,7]])
c = np.array([9,2])

In [23]:
C.transpose()

array([[ 2, -2],
       [ 3,  4],
       [ 4,  6],
       [-3,  7]])

In [30]:
C = np.random.randint(-2,2, size=(10, 100))

In [13]:
H,U = hermite_normal_form(C)

In [14]:
U

array([[  17.,   -3.,   -1.,    6.],
       [-305.,   51.,   10., -106.],
       [ 216.,  -36.,   -7.,   75.],
       [  -6.,    1.,    0.,   -2.]])

In [19]:
U2  = np.array([[5,21,19,33],[-1,-5,-6,-8],[0,0,1,0],[2,9,8,14]])

In [15]:
H

array([[1, 0, 0, 0],
       [0, 1, 0, 0]])

The next function calculates all solutions to a system of Diophantine equations. It takes a Herite normal form $H$ of $C$, the corresponding unimodular matrix $U$ with $CU=H$, and the right hand side vector $c$ as an input. 

In [20]:
def get_diophantine_solutions(H,U,c):
    q,m = H.shape
    B = H[:,0:H.shape[0]]
    Binv = np.linalg.inv(B)
    sol = np.dot(Binv,c)
    if(all(sol- np.floor(sol) == 0)):
        x_0 = np.append(sol,np.zeros(m-q))
        V_0 = np.concatenate([np.zeros([q,m-q]),np.eye(m-q)], axis = 0)
        V = np.dot(U,V_0)
        x = np.dot(U,x_0)
    else:
        x = []
        V = []
    return x,V

All solutions to our example thus correspond to

In [28]:
x,V = get_diophantine_solutions(H,U,c)
print('x_0 =')
print(x)
print('V =')
print(V)

x_0 =
[  147. -2643.  1872.   -52.]
V =
[[  -1.    6.]
 [  10. -106.]
 [  -7.   75.]
 [   0.   -2.]]


In [30]:
x,V = get_diophantine_solutions(H,U2,c)
print('x_0 =')
print(x)
print('V =')
print(V)

x_0 =
[ 87. -19.   0.  36.]
V =
[[ 19.  33.]
 [ -6.  -8.]
 [  1.   0.]
 [  8.  14.]]


Indeed, as we demonstrate below, $x$ solves the system $Cx = c$, and the columns of $V$ span $\{y| Cy=0\}$. 

In [31]:
print(np.dot(C,x))
print(np.dot(C,V))

[ 9.  2.]
[[ 0.  0.]
 [ 0.  0.]]


## Literature
[1] Theory Of Linear And Integer Programming, Alexander Schrijver, 

[2] Integer Programming and Algorithmic Geometry of Numbers, Friedrich Eisenbrand